In [ ]:
  import random
  import json
  import pickle
  import numpy as np
  import nltk
  from nltk.stem import WordNetLemmatizer
  from tensorflow.keras.models import Sequential
  from tensorflow.keras.layers import Dense,Activation, Dropout
  from tensorflow.keras.optimizers import SGD

In [ ]:
import nltk
nltk.download('punkt')
nltk.download('wordnet')
nltk.download('omw-1.4')
nltk.download('punkt_tab')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!
[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!


True

In [ ]:
lemmatizer = WordNetLemmatizer()
try:
    with open("/content/sample_data/intens.json", encoding='latin-1') as f:
        intents = json.load(f)
except UnicodeDecodeError:
    try:
        with open("/content/sample_data/intens.json", encoding='cp1252') as f:
            intents = json.load(f)
    except UnicodeDecodeError:
        print("Could not decode the file with either 'latin-1' or 'cp1252' encoding. Please check the file's encoding.")
        raise

words = []
classes = []
documents = []
ignore_letters = ['?','!','.','.']
words = [lemmatizer.lemmatize(word) for word in words if word not in ignore_letters]

In [ ]:
print(intents.keys())

dict_keys(['intents'])


In [ ]:
for intent in intents['intents']:
    for pattern in intent['patterns']:
        word_list = nltk.word_tokenize(pattern.lower())
        words.extend(word_list)
        documents.append((word_list, intent['tag']))
        if intent['tag'] not in classes:
            classes.append(intent['tag'])

words = [lemmatizer.lemmatize(word.lower()) for word in words if word not in ignore_letters]  # Added lower()
words = sorted(set(words))
classes = sorted(set(classes))
pickle.dump(words,open('words.pkl','wb'))
pickle.dump(classes,open('classes.pkl','wb'))

In [ ]:
print(words)

["'m", "'re", "'s", '911', 'a', 'about', 'abroad', 'academic', 'accident', 'active', 'activity', 'adios', 'adopting', 'adventure', 'advice', 'affair', 'afternoon', 'age', 'aid', 'alert', 'alright', 'am', 'amazing', 'ambition', 'ambivalent', 'an', 'analysis', 'and', 'angry', 'animal', 'annoying', 'anxiety', 'anxious', 'any', 'anyone', 'appreciate', 'appreciated', 'apps', 'are', 'artist', 'ask', 'asleep', 'assist', 'assistance', 'at', 'attorney', 'auto', 'average', 'awake', 'awesome', 'awful', 'baby', 'bad', 'baking', 'balanced', 'be', 'beauty', 'become', 'beginner', 'behavior', 'being', 'bender', 'best', 'better', 'big', 'bill', 'birth', 'birthday', 'blessed', 'block', 'board', 'book', 'born', 'bos', 'bot', 'brain', 'breakdown', 'breaking', 'breakup', 'briefing', 'brilliant', 'bring', 'broke', 'budget', 'budgeting', 'bunch', 'business', 'but', 'buy', 'buying', 'by', 'bye', 'ca', 'call', 'calm', 'can', 'capability', 'car', 'cardio', 'care', 'career', 'cash', 'cat', 'catch', 'challenge', 

In [ ]:
training = []
output_empty = [0] * len(classes)

for doc in documents:
    bag = []
    pattern_words = [lemmatizer.lemmatize(word.lower()) for word in doc[0]]
    for w in words:
        bag.append(1) if w in pattern_words else bag.append(0)

    output_row = list(output_empty)
    output_row[classes.index(doc[1])] = 1
    training.append([bag, output_row])

random.shuffle(training)
training = np.array(training, dtype=object)

train_x = np.array([x[0] for x in training])
train_y = np.array([x[1] for x in training])


In [ ]:
model = Sequential()
model.add(Dense(128,input_shape=(len(train_x[0]),), activation = 'relu'))
model.add(Dropout(0.5))
model.add(Dense(64, activation = 'relu'))
model.add(Dropout(0.5))
model.add(Dense(len(train_y[0]), activation = 'softmax'))
sgd = SGD(learning_rate=0.01, decay=1e-6, momentum=0.9, nesterov=True)
model.compile(loss='categorical_crossentropy',optimizer = sgd, metrics=['accuracy'])

In [ ]:
hist = model.fit(np.array(train_x), np.array(train_y), epochs=200,batch_size =5,verbose=1)
model.save('chatbotmodel.h5')

Epoch 1/200
229/229 ━━━━━━━━━━━━━━━━━━━━ 2s 4ms/step - accuracy: 0.0197 - loss: 4.0611
Epoch 2/200
229/229 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - accuracy: 0.0439 - loss: 4.0315
Epoch 3/200
229/229 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - accuracy: 0.0667 - loss: 3.9519
Epoch 4/200
229/229 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.1065 - loss: 3.8021
Epoch 5/200
229/229 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - accuracy: 0.1706 - loss: 3.4930
Epoch 6/200
229/229 ━━━━━━━━━━━━━━━━━━━━ 2s 6ms/step - accuracy: 0.2478 - loss: 3.1215
Epoch 7/200
229/229 ━━━━━━━━━━━━━━━━━━━━ 2s 3ms/step - accuracy: 0.3129 - loss: 2.8234
Epoch 8/200
229/229 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.3671 - loss: 2.5379
Epoch 9/200
229/229 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - accuracy: 0.4330 - loss: 2.2463
Epoch 10/200
229/229 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.4715 - loss: 2.0068
Epoch 11/200
229/229 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - accuracy: 0.5109 - loss: 1.8080
Epoch 12/200
229/229 ━━━━━━━━━━━━━━━━━━━━

In [ ]:
from tensorflow.keras.models import load_model

lemmatizer = WordNetLemmatizer()
intents = json.loads(open('/content/sample_data/intens.json').read())
words = pickle.load(open('words.pkl','rb'))
classes = pickle.load(open('classes.pkl','rb'))
model = load_model('/content/chatbotmodel.h5')

In [ ]:
def clean_up_sentence(sentence):
  sentence_words=nltk.word_tokenize(sentence)
  sentence_words = [lemmatizer.lemmatize(word) for word in   sentence_words]
  return sentence_words

In [ ]:
def bag_of_words(sentence):
    sentence_words =clean_up_sentence(sentence)
    bag = [0] * len(words)
    for w in sentence_words:
     for i , word in enumerate(words):
       if word ==w:
         bag[i] =1
    return np.array(bag)

In [ ]:
def predict_class(sentence):
    sentence_lower = sentence.lower()
    for intent in intents['intents']:
        if intent['tag'] == 'greeting':
            if any(sentence_lower == pattern.lower() for pattern in intent['patterns']):
                return [{'intent': 'greeting', 'probability': 1.0}]
    p = bag_of_words(sentence)
    res = model.predict(np.array([p]), verbose=0)[0]
    ERROR_THRESHOLD = 0.1
    results = [[i, r] for i, r in enumerate(res) if r > ERROR_THRESHOLD]
    results.sort(key = lambda x: x[1],reverse = True)
    return_list =[]
    if not results:
        max_index = np.argmax(res)
        results = [[max_index, res[max_index]]]

    results.sort(key=lambda x: x[1], reverse=True)
    return_list = []
    for r in results:
        return_list.append({'intent': classes[r[0]], 'probability': str(r[1])})
    return return_list

In [ ]:
def get_response(intents_list,intents_json):
    try:
        tag= intents_list[0]['intent']
        list_of_intents = intents_json['intents']
        for i in list_of_intents:
            if i['tag'] == tag:
               result = random.choice(i['responses'])
               break
    except IndexError:
         result = "I don't understand!"
    return result

In [ ]:
print("Bot hazır!")
while True:
    message = input('Sen: ')
    if message.lower() == 'çıkış':
        break

    ints = predict_class(message)
    res = get_response(ints, intents)
    print("Bot:", res)

Bot hazır!
Sen: hello
Bot: Greetings! How may I assist you?
Sen: hi 
Bot: Hello! How can I help you today?
Sen: how are you
Bot: Hey! What brings you here today?
Sen: who are you
Bot: I'm a simple AI chatbot designed to help you with various tasks.
Sen: tell me a music
Bot: Streaming services often have great discovery playlists based on your tastes.
Sen: i am sad
Bot: Happiness often comes from small daily practices - gratitude, connection, and engaging activities.
Sen: i wan to happy
Bot: Happiness often comes from small daily practices - gratitude, connection, and engaging activities.
Sen: i am sad
Bot: Helping others, spending time in nature, and practicing gratitude can boost happiness.
Sen: i want to happy
Bot: It's okay to feel down sometimes. Be patient with yourself and focus on small positive moments.


KeyboardInterrupt: Interrupted by user